1. Instalación de librerias

In [2]:
%pip install -r ../requirements.txt

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -------- ------------------------------- 0.3/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 6.9 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.


2. Importanción de dependencias y configuraciones


In [3]:
import sys
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os

# Ruta principal para poder importar la carpeta 'tools'
sys.path.append('..') 

from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent 

# Importamos las herramientas
from tools.banking_tools import obtener_saldo, obtener_gastos_recientes, analizar_estadisticas_periodo


3. Generamos nuestro Dataset con datos ficticios para poder realizar las pruebas

In [4]:
# Asegurarnos de que exista la carpeta data
os.makedirs('../data', exist_ok=True)

# Configuramos la generación de datos
np.random.seed(42)
fecha_fin = datetime(2026, 2, 27)
fecha_inicio = fecha_fin - timedelta(days=180) # 6 meses atrás

descripciones_y_categorias = [
    ("Supermercado Hipermaxi", "Alimentación", 150, 600),
    ("Pollos Copacabana", "Alimentación", 30, 100),
    ("Mercado Rodriguez", "Alimentación", 50, 200),
    ("Mi Teleférico", "Transporte", 3, 15),
    ("Minibus / Trufi", "Transporte", 2, 10),
    ("Pago Servicio ENTEL", "Servicios Básicos", 100, 200),
    ("Factura CRE / Delapaz", "Servicios Básicos", 80, 250),
    ("Cine Center", "Ocio", 50, 150),
    ("Suscripción Netflix", "Ocio", 55, 60),
    ("Farmacorp", "Salud", 20, 300),
    ("Transferencia QR - Amigo", "Otros", 20, 200),
    ("Cafetería Typica", "Gasto Hormiga", 25, 60),
    ("Snacks Kiosco", "Gasto Hormiga", 5, 20)

]

datos = []
fecha_actual = fecha_inicio

# Generamos entre 1 y 3 transacciones por día
while fecha_actual <= fecha_fin:
    num_transacciones = random.randint(1, 3)
    for _ in range(num_transacciones):
        desc, cat, min_val, max_val = random.choice(descripciones_y_categorias)
        monto = round(random.uniform(min_val, max_val), 2)
        datos.append({
            "id_cliente": "1001",
            "fecha": fecha_actual.strftime("%Y-%m-%d"),
            "monto": monto,
            "descripcion": desc,
            "categoria": cat # En un sistema real, el LLM clasificaría esto la primera vez
        })
    fecha_actual += timedelta(days=1)

# Creamos el DataFrame y lo guardamos como CSV
df_gastos = pd.DataFrame(datos)
ruta_csv = '../data/gastos_historicos.csv'
df_gastos.to_csv(ruta_csv, index=False)

print(f"✅ Se generaron {len(df_gastos)} transacciones.")
print(f"✅ Archivo guardado en: {ruta_csv}")
display(df_gastos.head())

✅ Se generaron 351 transacciones.
✅ Archivo guardado en: ../data/gastos_historicos.csv


,id_cliente,fecha,monto,descripcion,categoria
0,1001,2025-08-31,5.49,Minibus / Trufi,Transporte
1,1001,2025-09-01,104.03,Transferencia QR - Amigo,Otros
2,1001,2025-09-02,55.56,Suscripción Netflix,Ocio
3,1001,2025-09-03,19.70,Snacks Kiosco,Gasto Hormiga
4,1001,2025-09-03,225.27,Farmacorp,Salud


4. Configuración de nuestro modelo

In [5]:
#Cargamos nuestro archivo .env para las variables de entorno
load_dotenv('../.env')

# Instanciamos nuestras herramientas en una lista
tools = [obtener_saldo, obtener_gastos_recientes, analizar_estadisticas_periodo]

# Instanciamos el modelo de Azure OpenAI
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    temperature=0.2,
    max_retries=2
)
print("Modelo Azure OpenAI configurado correctamente.")


Modelo Azure OpenAI configurado correctamente.


5. Instrucciones para nuestro modelo

In [5]:
from datetime import datetime

fecha_hoy = datetime.now().strftime("%Y-%m-%d")

# 2. Definimos las instrucciones del sistema (System Prompt)
instrucciones = f"""Eres FinancIA, un asesor financiero personal empático, analítico y directo.
REGLA DE ORO TEMPORAL: Hoy es la fecha {fecha_hoy}. Usa esta fecha exacta como tu punto de referencia para calcular "esta semana", "últimos 7 días", "este mes", etc.

Tu objetivo es ayudar al usuario a entender sus finanzas, categorizar sus gastos y mejorar su capacidad de ahorro.

Sigue estas reglas estrictamente:
1. Usa las herramientas proporcionadas para consultar el saldo y el historial de gastos del cliente cuando te dé su ID.
2. Analiza el historial y agrupa los gastos por categorías semánticas (ej. Alimentación, Transporte, Servicios Básicos, Ocio/Entretenimiento, Salud).
3. Identifica claramente cuáles podrían ser "gastos hormiga" o gastos innecesarios que el usuario podría reducir.
4. Comunícate de forma clara y amigable, usando la moneda local (Bs.).
5. Termina tu intervención con 3 consejos o mas accionables y específicos basados en sus hábitos de consumo recientes si así lo consideras necesario.
6. ADVERTENCIA DE SEGURIDAD: Nunca recomiendes inversiones específicas ni asumas riesgos por el usuario.
"""

# 3. Creamos el agente usando LangGraph
agent_executor = create_react_agent(llm, tools, prompt=instrucciones)

# 4. Hacemos la prueba con nuestro cliente 1001
pregunta_usuario = "Hola FinancIA soy el cliente con el ID 1001, ¿cuánto dinero he gastado en total en los últimos 7 días y cuál es la categoría en la que más gasté?"

# Ejecutamos el agente pasándole la pregunta como un mensaje humano
respuesta = agent_executor.invoke({"messages": [HumanMessage(content=pregunta_usuario)]})

# Imprimimos la respuesta final
print("\n--- RESPUESTA FINAL DEL AGENTE ---\n")
print(respuesta["messages"][-1].content)

C:\Users\ldbar\AppData\Local\Temp\ipykernel_24016\26359699.py:21: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent_executor = create_react_agent(llm, tools, prompt=instrucciones)



--- RESPUESTA FINAL DEL AGENTE ---

En los últimos 7 días, has gastado un total de **1685.19 Bs.**. La categoría en la que más gastaste fue **Alimentación**, con un total de **813.63 Bs.**. 

Si necesitas más detalles o recomendaciones para optimizar tus gastos, no dudes en pedírmelo. 😊
